<a href="https://colab.research.google.com/github/shivakerur99/portfolio/blob/master/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [1]:
from vllm import LLM, SamplingParams
import torch
import pandas as pd
import time
import os
# from prompts import questions
questions = [
    "What is the capital of France?",
    "Who wrote 'Romeo and Juliet'?",
    "What is the meaning of life?",
    # Add more questions as needed
]

llm = LLM(
    "TheBloke/Mistral-7B-v0.1-AWQ",
    dtype=torch.float16,
    quantization="AWQ",
    )
from rich.progress import track

tensor_parallel_size = int(os.environ.get("DEVICES", "1"))

sampling_params = SamplingParams(max_tokens=200)


WARNING 03-03 23:05:33 config.py:618] Casting torch.bfloat16 to torch.float16.
WARNING 03-03 23:05:33 config.py:193] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 03-03 23:05:33 llm_engine.py:87] Initializing an LLM engine with config: model='TheBloke/Mistral-7B-v0.1-AWQ', tokenizer='TheBloke/Mistral-7B-v0.1-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 03-03 23:05:40 weight_utils.py:163] Using model weights format ['*.safetensors']
INFO 03-03 23:06:21 llm_engine.py:357] # GPU blocks: 2122, # CPU blocks: 2048
INFO 03-03 23:06:24 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-03 23:06:24 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-03 23:06:40 model_runner.py:756] Graph capturing finished in 16 secs.


In [3]:
results = []
for q in questions:
    t0 = time.perf_counter()
    output = llm.generate(q, sampling_params=sampling_params)[0]
    t1 = time.perf_counter()
    results.append(
        {"time": t1 - t0, "tokens_generated": len(output.outputs[0].token_ids)}
    )

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.61s/it]


In [2]:
from vllm import SamplingParams

sampling_params=SamplingParams(max_tokens=200, temperature=0)
print(llm.generate("The weather today is very cold. You should take", sampling_params=sampling_params))

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.42s/it]

[RequestOutput(request_id=0, prompt='The weather today is very cold. You should take', prompt_token_ids=[1, 415, 8086, 3154, 349, 1215, 5256, 28723, 995, 1023, 1388], prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' care of your health.\n\nThe weather today is very cold. You should take care of your health.\n\nThe weather today is very cold. You should take care of your health.\n\nThe weather today is very cold. You should take care of your health.\n\nThe weather today is very cold. You should take care of your health.\n\nThe weather today is very cold. You should take care of your health.\n\nThe weather today is very cold. You should take care of your health.\n\nThe weather today is very cold. You should take care of your health.\n\nThe weather today is very cold. You should take care of your health.\n\nThe weather today is very cold. You should take care of your health.\n\nThe weather today is very cold. You should take care of your health.\n\nThe weather today is very

In [4]:
import pandas as pd

df = pd.DataFrame(results)
df.to_csv(f"vllm-benchmark-2-GPUs.csv", index=False)

In [12]:
df["tokens_per_sec"] = df.tokens_generated / df.time

# New Section

In [5]:
def benchmark(csv_path):
    df = pd.read_csv(csv_path)
    n = len(df)
    print(f"Average time to complete {n} texts: {df.time.mean(): .3f}")
    print(f"Average output tokens: {df.tokens_generated.mean(): .3f}")
    df["tokens_per_sec"] = df.tokens_generated / df.time
    print(f"Average tokens/sec: {df.tokens_per_sec.mean(): .3f}")
    return df

benchmark("/content/vllm-benchmark-2-GPUs.csv")

Average time to complete 3 texts:  6.563
Average output tokens:  200.000
Average tokens/sec:  30.481


,time,tokens_generated,tokens_per_sec
0,6.447328,200,31.020603
1,6.618681,200,30.217499
2,6.621531,200,30.204496


In [14]:
!pip install transformers torch
